# **TF-IDF**
7,395 개의 웹사이트 데이터이며 각각 27개의 feature(column) 정보를 가지고 있다. 

y 예측할 정답컬럼 : label, 각각의 사이트가 환경친화적 사이트면 1, 아니면 0이다. 

X 모델 입력 컬럼 : boilerplate 컬럼 하나만 사용한다. 웹사이트의 content 텍스트 정보를 가지고 있다. 

즉 boilerplate 컬럼만 TF-IDF 방법으로 분석해서 모델의 입력으로 넣어주는 방식을 연구해보자! 

## **데이터 불러오기 & import**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os
path = "gdrive/My Drive/Colab Notebooks/02_Test/data/"
os.listdir(path)

['stumble_upon_evergreen.tsv']

In [ ]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline # 전처리 + 모델링 
from nltk.corpus import words

## **데이터 살펴보기**

In [ ]:
df = pd.read_table(path + "stumble_upon_evergreen.tsv")
df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0,0.024908,0,0.228887,0.050473,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [ ]:
df.columns

Index(['url', 'urlid', 'boilerplate', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio', 'label'],
      dtype='object')

In [ ]:
df.shape

(7395, 27)

## **데이터 전처리 Data Preprocessing** 
boilerplate column을 살펴 본 결과 "title", "body", "url을 기준으로 나누어야 하며 중간 중간 ":" 문자열을 제거해줄 필요가 있습니다. 


*   {} 제거하기 
*   title, body, url 문자열을 기준으로 분할하기 
*   분할 후 필요없는 뒤 문자열 제거 

의 과정이 필요합니다. 





In [285]:
X_column = ['boilerplate']
Y_column = 'label'

input_data = df[X_column + [Y_column]].copy() 
input_data.head().values

array([['{"title":"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries","body":"A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest provider of computer services looks to Silicon Valley for input gleaning many ideas from its Almaden research center in 

In [286]:
input_data['boilerplate']=input_data['boilerplate'].str.replace("[{}]",'')
#input_data.head().values

In [287]:
input_data[['temp', 'title']] = input_data['boilerplate'].str.split("\"title\":", expand=True)
input_data = input_data.drop(['temp'], axis=1)

input_data.head()

,boilerplate,label,title
0,"""title"":""IBM Sees Holographic Calls Air Breath...",0,"""IBM Sees Holographic Calls Air Breathing Batt..."
1,"""title"":""The Fully Electronic Futuristic Start...",1,"""The Fully Electronic Futuristic Starting Gun ..."
2,"""title"":""Fruits that Fight the Flu fruits that...",1,"""Fruits that Fight the Flu fruits that fight t..."
3,"""title"":""10 Foolproof Tips for Better Sleep "",...",1,"""10 Foolproof Tips for Better Sleep "",""body"":""..."
4,"""title"":""The 50 Coolest Jerseys You Didn t Kno...",0,"""The 50 Coolest Jerseys You Didn t Know Existe..."


In [288]:
input_data[['temp', 'body']] = input_data['title'].str.split(",\"body\":", expand=True)
input_data = input_data.drop(['temp'], axis=1)

input_data.head()

,boilerplate,label,title,body
0,"""title"":""IBM Sees Holographic Calls Air Breath...",0,"""IBM Sees Holographic Calls Air Breathing Batt...","""A sign stands outside the International Busin..."
1,"""title"":""The Fully Electronic Futuristic Start...",1,"""The Fully Electronic Futuristic Starting Gun ...","""And that can be carried on a plane without th..."
2,"""title"":""Fruits that Fight the Flu fruits that...",1,"""Fruits that Fight the Flu fruits that fight t...","""Apples The most popular source of antioxidant..."
3,"""title"":""10 Foolproof Tips for Better Sleep "",...",1,"""10 Foolproof Tips for Better Sleep "",""body"":""...","""There was a period in my life when I had a lo..."
4,"""title"":""The 50 Coolest Jerseys You Didn t Kno...",0,"""The 50 Coolest Jerseys You Didn t Know Existe...","""Jersey sales is a curious business Whether yo..."


In [289]:
input_data[['temp', 'url']] = input_data['body'].str.split(",\"url\":", expand=True)
input_data = input_data.drop(['temp'], axis=1)

input_data.head()

,boilerplate,label,title,body,url
0,"""title"":""IBM Sees Holographic Calls Air Breath...",0,"""IBM Sees Holographic Calls Air Breathing Batt...","""A sign stands outside the International Busin...","""bloomberg news 2010 12 23 ibm predicts hologr..."
1,"""title"":""The Fully Electronic Futuristic Start...",1,"""The Fully Electronic Futuristic Starting Gun ...","""And that can be carried on a plane without th...","""popsci technology article 2012 07 electronic ..."
2,"""title"":""Fruits that Fight the Flu fruits that...",1,"""Fruits that Fight the Flu fruits that fight t...","""Apples The most popular source of antioxidant...","""menshealth health flu fighting fruits cm mmc ..."
3,"""title"":""10 Foolproof Tips for Better Sleep "",...",1,"""10 Foolproof Tips for Better Sleep "",""body"":""...","""There was a period in my life when I had a lo...","""dumblittleman 2007 12 10 foolproof tips for b..."
4,"""title"":""The 50 Coolest Jerseys You Didn t Kno...",0,"""The 50 Coolest Jerseys You Didn t Know Existe...","""Jersey sales is a curious business Whether yo...","""bleacherreport articles 1205138 the 50 cooles..."


In [290]:
input_data['title'] = input_data['title'].str.split(",").str[0]
input_data['body'] = input_data['body'].str.split(",").str[0]
input_data

,boilerplate,label,title,body,url
0,"""title"":""IBM Sees Holographic Calls Air Breath...",0,"""IBM Sees Holographic Calls Air Breathing Batt...","""A sign stands outside the International Busin...","""bloomberg news 2010 12 23 ibm predicts hologr..."
1,"""title"":""The Fully Electronic Futuristic Start...",1,"""The Fully Electronic Futuristic Starting Gun ...","""And that can be carried on a plane without th...","""popsci technology article 2012 07 electronic ..."
2,"""title"":""Fruits that Fight the Flu fruits that...",1,"""Fruits that Fight the Flu fruits that fight t...","""Apples The most popular source of antioxidant...","""menshealth health flu fighting fruits cm mmc ..."
3,"""title"":""10 Foolproof Tips for Better Sleep "",...",1,"""10 Foolproof Tips for Better Sleep ""","""There was a period in my life when I had a lo...","""dumblittleman 2007 12 10 foolproof tips for b..."
4,"""title"":""The 50 Coolest Jerseys You Didn t Kno...",0,"""The 50 Coolest Jerseys You Didn t Know Existe...","""Jersey sales is a curious business Whether yo...","""bleacherreport articles 1205138 the 50 cooles..."
...,...,...,...,...,...
7390,"""title"":""Kno Raises 46 Million More To Build M...",0,"""Kno Raises 46 Million More To Build Most Powe...","""Marc Andreessen is normally enthusiastic abou...","""techcrunch 2010 09 08 kno raises 46 million m..."
7391,"""title"":""Why I Miss College "",""body"":""Mar 30 2...",0,"""Why I Miss College ""","""Mar 30 2009 I d like to congratulate Jane on ...","""uncoached category why i miss college"""
7392,"""title"":""Sweet Potatoes Eat This Not That i'm...",1,"""Sweet Potatoes Eat This Not That i'm eating ...","""They re loaded with vitamin C which smoothes ...","""eatthis menshealth slide sweet potatoes slide..."
7393,"""title"":""Naturally Ella "",""body"":"" "",""url"":""na...",1,"""Naturally Ella """,""" ""","""naturallyella"""


In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(
    input_data[X_column], # X 입력 데이터
    input_data[Y_Column], # Y 정답 데이터, label 데이터 
    test_size = 0.2,
    #train_size = 13, 
    shuffle=True,
    random_state=42)

# **TF-IDF (Term Frequency - Inverse Document Frequency)**


*   **TF(단어 빈도, term frequency)**는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값. 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다.
*   하지만 하나의 문서에서 많이 나오지 않고 다른 문서에서 자주 등장하면 단어의 중요도는 낮아진다.
*   **DF(문서 빈도, document frequency)**라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다.
*   TF-IDF는 TF와 IDF를 곱한 값으로 점수가 높은 단어일수록 다른 문서에는 많지 않고 해당 문서에서 자주 등장하는 단어를 의미한다.





In [291]:
vectorizer = CountVectorizer(analyzer = 'train_data', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )
vectorizer

LookupError: ignored